In [64]:
### Installing packages
# pip install selenium
# pip install webdrive_manager
# pip install yfinance

# Import packages
import yfinance as yf
import requests
import pandas as pd
from datetime import datetime

from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

### Strategy

The stock screening strategy below is designed to complement the VWAP and ABCD pattern day trading strategies outlined in the book "Day Trading for a Living" by

#### Pre-Market Gappers
Criteria for the Gappers Scanner includes:
- Stocks that in the pre-market gapped up or down at least 2%
- Stocks that have traded at least 100,000 shares by 9am in pre-market
- Stocks that have an average daily volume of over 500,000 shares
- Stocks that have Average True Range (ATR) of at least 50 cents
- There is a fundamental catalyst for the stock

#### Additional Filter
We will filter for only Medium Float stocks (20 to 500 million shares) with a price range between $10-$100 and Large float stocks (500+ million shares) with a price range > $20. These stock work best with the VWAP and ABCD pattern trading strategies.

#### Human Screen
I will review the output manually and do my own research into whether there is a fundamental catalyst for the stock


In [49]:
# Initialising Selenium and setting options
options = webdriver.ChromeOptions() 
options.headless = True 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) 

In [50]:
# Lets begin by webscraping a list of the top 100 losers and top 100 gainers for the day

# Gainers
t100_gainers_url = "https://finance.yahoo.com/screener/predefined/day_gainers?count=100&offset=0"

# Losers
t100_losers_url = "https://finance.yahoo.com/screener/predefined/day_losers?count=100&offset=0"

In [51]:
# Function to scrape list stock symbols from Yahoo Finance's screener list
def scrape_gappers(url, driver):
    # Load page
	driver.get(url)
    
	# Find row element
	row_eles = driver.find_elements(By.CLASS_NAME, 'simpTblRow')

	# In each row we are interested in the Ticker Symbol and Price. 
	# Although we will also be filtering for Volume and Float (Market Cap/Price), we will do this at a later stage once we search for the ticker symbols using the yfinance library. This will save us the trouble of having to parse text with B and M representing billions and millions.
	gappers_list = list(map(row_to_tuple, row_eles))
	return gappers_list

# Function to convert Yahoo Finance row element to a tuple of (Ticker Symbol, Price)
def row_to_tuple(row_ele):
	# Getting the Ticker Symbol
	symbol_ele = row_ele.find_element(By.XPATH,'td[@aria-label="Symbol"]')
	symbol_str = symbol_ele.text

	# Getting the Price
	price_ele = row_ele.find_element(By.XPATH,'td[@aria-label="Price (Intraday)"]')
	price_num = float(price_ele.text)

	# Returning data as a tuple - (Ticker Symbol: String, Price: Float)
	return (symbol_str, price_num)

In [52]:
# Stock ticker symbols
t100_gainers_all = scrape_gappers(t100_gainers_url, driver)
t100_losers_all = scrape_gappers(t100_losers_url, driver)

### One criteria of the screener is that the price much be above $10. Lets filter these down based on this.

# Function to filter Stocks with prices > $10 and return only the Ticker Symbol
def stocks_above_10(sym_pri_tuple_lst):
	SYMBOL_TUPLE_INDEX = 0
	PRICE_TUPLE_INDEX = 1

	stocks_above_10 = []

	for tuple in sym_pri_tuple_lst:
		price = tuple[PRICE_TUPLE_INDEX]

		if (price > 10):
			symbol = tuple[SYMBOL_TUPLE_INDEX]
			stocks_above_10.append(symbol)
	
	return stocks_above_10

# Filtered Lists
t100_gainers = stocks_above_10(t100_gainers_all)
t100_losers = stocks_above_10(t100_losers_all)

print(t100_gainers)
print(t100_losers)

['HCM', 'BGNE', 'NAAS', 'ATAT', 'AI', 'DLO', 'KRYS', 'LEGN', 'COLB', 'DEA', 'CTLT', 'LKNCY', 'CFLT', 'FLNC', 'LESL', 'ALNY', 'ITCI', 'SYM', 'WAL', 'BUR', 'ROKU', 'MDGL', 'SYIEY', 'FRC', 'FOXF', 'RLAY', 'BABA', 'IMCR', 'TECK', 'ENV', 'RXDX', 'RYAAY', 'EWBC', 'EDR', 'ARGX', 'GOOGL', 'TTD', 'GOOG', 'PARAA', 'MRTX', 'DOCS', 'SRPT', 'ARWR', 'HKXCY', 'MNDY', 'GDS', 'CMA', 'SUPN', 'BPMC', 'PINS', 'DRS', 'SHTDY', 'HTHT', 'IAS', 'HSHCY', 'APPN', 'NEWR', 'TAK', 'FIS', 'PARA', 'BMRN', 'VIPS', 'BDRBF', 'SRAD', 'MPNGY', 'VIR']
['LEVI', 'LITE', 'PBI-PB', 'FN', 'SMCI', 'ZGN', 'MOS', 'CVCO', 'TTNDF', 'ABNB', 'NTR', 'GTLS', 'WIRE', 'VRT', 'ATKR', 'FMC', 'PBF', 'CPRI', 'UNVGY', 'SIG', 'RPM', 'KBH', 'SPWR', 'ARCH', 'KMTUY', 'BDC', 'SWAV', 'CRK', 'FRO', 'DKILY', 'WCC', 'ARW', 'MDC', 'FJTSY', 'CVE', 'CEQP', 'OLK', 'PCVX', 'DNUT', 'MPC', 'WIX', 'LSRCY', 'AIT', 'PEN', 'LECO', 'SANM', 'NEOG', 'PVH', 'LRLCY', 'BE', 'LGIH', 'TOELY', 'YPF', 'KTB', 'CIEN', 'MTSI', 'PKX']


In [53]:
# Great! Now that we have our list of gapping stocks above $10, lets get more detailed information about these with the yfinance package.
# We want information in the following data structure (Ticker Symbol: String, Price: float, Change: float, AverageVolume: int, Float: float, AverageTrueRange: float)

# Function to get the latest price and average true range
def stock_price_atr(tickerdata):
    hist = tickerdata.history(interval='5m')
    price = hist.iloc[-1,3]
    
	# We will use a 14 period price range on the 5 min interval chart to determine the average true range
    hist['Range'] = hist['Close'].rolling(14).apply(lambda x: abs(x.iloc[13] - x.iloc[0]))
    # Because the trading day is from 9:30AM to 4:00PM, there are total of 78 5 Min intervals. We will use this to calculate the average 14 period 5-min range
    NUM_5_MIN_INTERVALS = 78
    avg_true_range= hist.tail(78)['Range'].mean()
    
    return price, avg_true_range

def stock_avg_vol(tickerdata):
	hist = tickerdata.history(interval='1d')
	return hist.tail(10)['Volume'].mean()
	
# Function to convert ticker symbol to (Ticker Symbol: String, Name: String, Price: float, Change: float, AverageVolume: int, Float: float, AverageTrueRange: float) data format
def tickers_to_data(tickers, data):
	result = []
        
	for ticker in tickers:
		ticker_data = data.tickers[ticker]

		### Getting data
		info = ticker_data.info

		name = info['longName']
		price, avg_true_range = stock_price_atr(ticker_data)
		change = info['regularMarketChangePercent']
		average_vol = stock_avg_vol(ticker_data)
		# Float Calculations
		market_cap = info['marketCap']
		ticker_float = market_cap/price

		### Apply Filters
		# Needs to be at least medium float (> 20 million shares)
		float_satisfied = ticker_float > 20,000,000
		# Needs to have an average volume of > 500,000
		avg_vol_satisfied = average_vol > 500,000
		# Needs to have a range of > $0.50
		range_satisfied = avg_true_range > 0.5

		if (float_satisfied and avg_vol_satisfied and range_satisfied):
			ticker_res = (ticker, name, price, change, average_vol, ticker_float, avg_true_range)
			result.append(ticker_res)
		
		# news = ticker_data.news

	return result
	


In [54]:
all_gappers = t100_gainers + t100_losers
data = yf.Tickers(all_gappers)

result = tickers_to_data(all_gappers, data)

print(result)

DRS: No data found for this date range, symbol may be delisted
[('BGNE', 'BeiGene, Ltd.', 238.10499572753906, 12.476414, 230860.0, 104616362.69280075, 4.545689509465144), ('AI', 'C3.ai, Inc.', 22.836299896240234, 8.29777, 40080010.0, 112228181.7827218, 0.5964205815241888), ('KRYS', 'Krystal Biotech, Inc.', 84.4000015258789, 7.5305758, 232330.0, 25760645.60062055, 1.0430126679249299), ('LEGN', 'Legend Biotech Corporation', 51.310001373291016, 6.4355454, 578670.0, 164938325.89147687, 0.5753437922551081), ('CTLT', 'Catalent, Inc.', 64.94999694824219, 5.40409, 1369990.0, 180089997.68423492, 0.7837925935402895), ('ALNY', 'Alnylam Pharmaceuticals, Inc.', 211.0, 5.0642357, 822830.0, 124218120.492891, 1.6744676247621193), ('ROKU', 'Roku, Inc.', 64.07009887695312, 4.58626, 11424380.0, 140130643.61337474, 0.7354590098063151), ('MDGL', 'Madrigal Pharmaceuticals, Inc.', 248.17999267578125, 4.578861, 281480.0, 18094150.264024153, 2.314168685521835), ('FOXF', 'Fox Factory Holding Corp.', 111.6800003

In [63]:
# Now lets pop this data into a pandas dataframe and export into excel

df = pd.DataFrame(result,columns=['Ticker', 'Name', 'Price', 'Change', 'Average Vol', 'Float', 'ATR'])

today = datetime.today().strftime('%d-%m-%Y')

df.to_csv(today + ' - gappers.xlsx')